In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

/coc/testnvme/pputta7/mambaforge/envs/lmnav/lib/python3.9/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


[2023-12-13 17:38:17,054] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [56]:
model.to("cuda")
seqs = ["Hello my name is", "What"]
inputs = tokenizer(seqs)

# pad with bos token on the left
torch.set_grad_enabled(False)

In [98]:
print(tokenizer.batch_decode(inputs['input_ids'], skip_special_tokens=False))
print(inputs['input_ids'])

['<s> Hello my name is', '<s> What']
[[1, 15043, 590, 1024, 338], [1, 1724]]


In [58]:
import torch
inps0 = torch.tensor(inputs['input_ids'][0])[None, :]
inps1 = torch.tensor(inputs['input_ids'][1])[None, :]
out0 = model(input_ids=inps0.cuda()).logits
out1 = model(input_ids=inps1.cuda()).logits

In [59]:
naive_batched_inp0 = inps0
naive_batched_inp1 = torch.nn.functional.pad(inps1, (3, 0), mode="constant", value=1)
naive_batched_inp = torch.cat([naive_batched_inp0, naive_batched_inp1], dim=0)
naive_batched_out = model(input_ids=naive_batched_inp.cuda()).logits

In [80]:
assert torch.allclose(out0[0], naive_batched_out[0], atol=1e-04, rtol=1e-05)
assert not torch.allclose(out1[0], naive_batched_out[1,3:], atol=1e-04, rtol=1e-05)

In [93]:
batched_inp0 = inps0
batched_inp1 = torch.nn.functional.pad(inps1, (3, 0), mode="constant", value=1)

pos_ids = torch.tensor([[0, 1, 2, 3, 4], [0, 0, 0, 1, 2]]).cuda()
attn_mask = torch.tensor([[1, 1, 1, 1, 1], [0, 0, 1, 1, 1]]).cuda()

batched_inp = torch.cat([batched_inp0, batched_inp1], dim=0)
batched_out = model(input_ids=naive_batched_inp.cuda(),
                   position_ids=pos_ids,
                   attention_mask=attn_mask).logits

In [94]:
assert torch.allclose(out0[0], batched_out[0], atol=1e-04, rtol=1e-05)
assert not torch.allclose(out1[0], batched_out[1,3:], atol=1e-04, rtol=1e-05)

In [95]:
out1[0]

tensor([[-12.9832,  -7.4134,  -0.4327,  ...,  -6.8297,  -8.0879,  -7.5863],
        [-12.6002,  -9.4469,  -2.6035,  ...,  -7.0415,  -9.5093,  -9.2344]],
       device='cuda:0')

In [96]:
batched_out[1, 3:]

tensor([[-12.9832,  -7.4134,  -0.4327,  ...,  -6.8297,  -8.0879,  -7.5863],
        [-12.2200,  -8.4223,  -2.0610,  ...,  -6.8798,  -9.4445,  -8.9009]],
       device='cuda:0')

In [85]:
tokenizer.eos_token_id

2